In [1]:
# possibe choices of data and dependent variable
# DEPENDENT VARIABLE
# A) final result (especially in case of modules without exam)
# - the result likely comes from weighted assignments score meaning that the behavioral variable derived
# from activity pattern may not work in the desired way and look just at intensity of the activity before assignments,
# which is fine, but it may be better if the measured outcome is not directly dependent on the assignments meaning
# that the short time effect of scoring fine on the assignment on the knowledge is not takes as the proxy for
# the overall student knowledge, which is represented better, by my lights, by the exam result
# B) exam score (preffered for modules with exam)

In [3]:
import pandas as pd

In [16]:
courses = pd.read_csv('../data/courses.csv')
assessments = pd.read_csv('../data/assessments.csv')
studentInfo = pd.read_csv('../data/studentInfo.csv')
studentAssessment = pd.read_csv('../data/studentAssessment.csv')

print('courses')
display(courses.head())
print('assessments')
display(assessments.head())
print('studentInfo')
display(studentInfo.head())
print('studentAssessment')
display(studentAssessment.head())

courses


,code_module,code_presentation,module_presentation_length
0,AAA,2013J,268
1,AAA,2014J,269
2,BBB,2013J,268
3,BBB,2014J,262
4,BBB,2013B,240


assessments


,code_module,code_presentation,id_assessment,assessment_type,date,weight
0,AAA,2013J,1752,TMA,19.0,10.0
1,AAA,2013J,1753,TMA,54.0,20.0
2,AAA,2013J,1754,TMA,117.0,20.0
3,AAA,2013J,1755,TMA,166.0,20.0
4,AAA,2013J,1756,TMA,215.0,30.0


studentInfo


,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result
0,AAA,2013J,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass
1,AAA,2013J,28400,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass
2,AAA,2013J,30268,F,North Western Region,A Level or Equivalent,30-40%,35-55,0,60,Y,Withdrawn
3,AAA,2013J,31604,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,Pass
4,AAA,2013J,32885,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,Pass


studentAssessment


,id_assessment,id_student,date_submitted,is_banked,score
0,1752,11391,18,0,78.0
1,1752,28400,22,0,70.0
2,1752,31604,17,0,72.0
3,1752,32885,26,0,69.0
4,1752,38053,19,0,79.0


In [94]:
# ------ (1) ------
# determining the code_module, code_presentation combinations with scored exam (and results of this exam)
def get_module_presentation_combination_with_exam():

    output = {}
    studentInfo_constrained = studentInfo.copy()
    
    for cm, cp in zip(courses[['code_module']].values, courses[['code_presentation']].values):
        code_module, code_presentation = cm[0], cp[0]
#         print(f'{code_module} {code_presentation}:')
        exam_id_assessment = assessments.set_index(['code_module', 'code_presentation', 'assessment_type']).loc[(code_module, code_presentation, 'Exam')]['id_assessment'].values[0]
#         print(exam_id_assessment)
            
#         studentAssessment_tmp = studentAssessment.copy()
        
        if exam_id_assessment in studentAssessment.id_assessment.unique():
            output[code_module] = output.get(code_module, [])+[code_presentation]
        
    return output

module_presentation_combination_with_exam = get_module_presentation_combination_with_exam()
# display(all_exam_id_assessment)

/tmp/ipykernel_18681/1018978835.py:11: PerformanceWarning: indexing past lexsort depth may impact performance.
  exam_id_assessment = assessments.set_index(['code_module', 'code_presentation', 'assessment_type']).loc[(code_module, code_presentation, 'Exam')]['id_assessment'].values[0]
/tmp/ipykernel_18681/1018978835.py:11: PerformanceWarning: indexing past lexsort depth may impact performance.
  exam_id_assessment = assessments.set_index(['code_module', 'code_presentation', 'assessment_type']).loc[(code_module, code_presentation, 'Exam')]['id_assessment'].values[0]
/tmp/ipykernel_18681/1018978835.py:11: PerformanceWarning: indexing past lexsort depth may impact performance.
  exam_id_assessment = assessments.set_index(['code_module', 'code_presentation', 'assessment_type']).loc[(code_module, code_presentation, 'Exam')]['id_assessment'].values[0]
/tmp/ipykernel_18681/1018978835.py:11: PerformanceWarning: indexing past lexsort depth may impact performance.
  exam_id_assessment = assessme

In [95]:
module_presentation_combination_with_exam

{'CCC': ['2014J', '2014B'], 'DDD': ['2013J', '2014J', '2013B', '2014B']}

In [103]:
all_exam_id_assessment = assessments.loc[assessments['assessment_type']=='Exam'].set_index(['code_module', 'code_presentation'])


# all_exam_id_assessment = all_exam_id_assessment.loc[assessments['code_presentation']=='2013J']

display(all_exam_id_assessment.loc[['CCC', 'DDD']])

id_assessment assessment_type   date  weight
code_module code_presentation                                              
CCC         2014B                      24290            Exam    NaN   100.0
            2014B                      40087            Exam    NaN   100.0
            2014J                      24299            Exam    NaN   100.0
            2014J                      40088            Exam    NaN   100.0
DDD         2013B                      25340            Exam  240.0   100.0
            2013J                      25354            Exam  261.0   100.0
            2014B                      25361            Exam  241.0   100.0
            2014J                      25368            Exam    NaN   100.0

In [150]:
def get_module_presentation_data(code_module,
                    code_presentation,
                    exam_id_assessment,
                    first_try=True,
                    drop_unregistered=True,
                    have_final_result=True,
                    if_exam_keep_written=True):
    
    studentInfo_tmp = studentInfo.copy()
#     filtering module
    studentInfo_tmp = studentInfo_tmp.loc[studentInfo_tmp.code_module==code_module]
#     filtering module presentation
    studentInfo_tmp = studentInfo_tmp.loc[studentInfo_tmp.code_presentation==code_presentation]
#     dropping multipe attempts
    studentInfo_tmp.drop(labels=studentInfo_tmp.loc[studentInfo_tmp.num_of_prev_attempts!=0].index,inplace=True)
#     dropping Withrawn
    studentInfo_tmp.drop(labels=studentInfo_tmp.loc[studentInfo_tmp.final_result=='Withdrawn'].index,inplace=True)
    studentInfo_tmp.drop(columns=['num_of_prev_attempts'], inplace=True)
    studentInfo_tmp = studentInfo_tmp.set_index(['id_student'])
    
    studentAssessment_tmp = studentAssessment.copy()
    studentAssessment_tmp = studentAssessment_tmp.set_index(['id_assessment','id_student']).loc[exam_id_assessment]
    studentAssessment_tmp.drop(columns=['is_banked'], inplace=True)
    
#     print(studentInfo_tmp.merge(studentAssessment_tmp, how='left', left_index=True, right_index=True).shape)
#     print(studentInfo_tmp.shape, studentAssessment_tmp.shape)
#     display(studentInfo_tmp)
#     display(studentAssessment_tmp)
    
    return studentInfo_tmp.merge(studentAssessment_tmp, how='left', left_index=True, right_index=True)

In [151]:
studentData_CCC_2014J = get_module_presentation_data('CCC','2014J',24299)

In [158]:
studentData_CCC_2014J.loc[studentData_CCC_2014J.score.isnull()].loc[studentData_CCC_2014J.final_result=='Pass'].index.values

array([  61227, 1777834])

In [156]:
studentAssessment.set_index(['id_student']).loc[61227]
# 61227

,id_assessment,date_submitted,is_banked,score
id_student,,,,
61227,24295,21,0,78.0
61227,24296,69,0,72.0
61227,24297,147,0,56.0
61227,24298,219,0,67.0
